<a href="https://colab.research.google.com/github/mariuszkr33/ML/blob/main/ReNewNew_04_04_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)

from sklearn.model_selection import cross_val_score, train_test_split, GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
#import scikitplot as skplt
#import eli5

import xgboost as xgb

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df_train = pd.read_csv(r'/content/drive/MyDrive/ReNew_Participants_Data/train.csv')
df_test = pd.read_csv(r'/content/drive/MyDrive/ReNew_Participants_Data/test.csv')

In [4]:
df_train.sample()

,timestamp,active_power_calculated_by_converter,active_power_raw,ambient_temperature,generator_speed,generator_winding_temp_max,grid_power10min_average,nc1_inside_temp,nacelle_temp,reactice_power_calculated_by_converter,reactive_power,wind_direction_raw,wind_speed_raw,wind_speed_turbulence,turbine_id,Target
755874,2021-09-26 13:50:00,33.491756,45.088872,34.854703,747.629873,55.508633,30.666582,34.390779,32.547852,1.406529,-0.480113,335.564026,3.415115,0.310881,Turbine_97,46.404673


In [5]:
df_train.shape

(909604, 16)

In [6]:
df_test.sample()

,timestamp,active_power_calculated_by_converter,active_power_raw,ambient_temperature,generator_speed,generator_winding_temp_max,grid_power10min_average,nc1_inside_temp,nacelle_temp,reactice_power_calculated_by_converter,reactive_power,wind_direction_raw,wind_speed_raw,wind_speed_turbulence,turbine_id
91941,2021-12-20 00:07:00,1300.364726,1330.582926,22.680983,1200.487244,59.856468,1262.01943,30.035873,27.223031,218.375659,267.377833,106.63045,8.711982,0.633757,Turbine_120


In [7]:
df_test.shape

(303202, 15)

In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909604 entries, 0 to 909603
Data columns (total 16 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   timestamp                               909604 non-null  object 
 1   active_power_calculated_by_converter    909604 non-null  float64
 2   active_power_raw                        909604 non-null  float64
 3   ambient_temperature                     909604 non-null  float64
 4   generator_speed                         909604 non-null  float64
 5   generator_winding_temp_max              909604 non-null  float64
 6   grid_power10min_average                 909604 non-null  float64
 7   nc1_inside_temp                         909604 non-null  float64
 8   nacelle_temp                            909604 non-null  float64
 9   reactice_power_calculated_by_converter  909604 non-null  float64
 10  reactive_power                          9096

In [9]:
df_train = df_train.sort_values(by=["turbine_id"])

In [10]:
df_turbine = df_train[ ["turbine_id", "Target"] ].groupby(["turbine_id"]).agg(["mean", "std", "median", "size"])["Target"].reset_index()
df_turbine.head()

,turbine_id,mean,std,median,size
0,Turbine_01,49.288458,3.342025,48.370728,54171
1,Turbine_10,47.186210,2.236599,47.576019,55089
2,Turbine_103,45.604759,1.819612,45.741870,56944
3,Turbine_105,46.585878,2.211376,46.940166,57341
4,Turbine_108,46.549006,1.834950,46.639037,57401


In [11]:
df_train_ext = pd.merge(df_train, df_turbine, on=["turbine_id"], how="left").fillna(-1)
df_test_ext = pd.merge(df_test, df_turbine, on=["turbine_id"], how="left").fillna(-1)
df_train_ext.head()

,timestamp,active_power_calculated_by_converter,active_power_raw,ambient_temperature,generator_speed,generator_winding_temp_max,grid_power10min_average,nc1_inside_temp,nacelle_temp,reactice_power_calculated_by_converter,reactive_power,wind_direction_raw,wind_speed_raw,wind_speed_turbulence,turbine_id,Target,mean,std,median,size
0,2021-03-26 00:27:00,317.745799,322.015981,34.871678,848.178396,67.905722,356.620722,31.263564,34.804127,104.915122,131.444270,123.696144,5.488043,0.392889,Turbine_01,56.621962,49.288458,3.342025,48.370728,54171
1,2021-08-22 05:33:00,247.187823,248.072744,31.406154,778.351400,56.940449,309.963074,29.224203,31.286025,27.074919,44.535753,319.350937,5.208137,0.452404,Turbine_01,47.564577,49.288458,3.342025,48.370728,54171
2,2021-05-23 19:49:00,149.306491,1196.192232,35.320692,6.103759,79.718864,1060.928284,32.114172,35.241397,-1.012690,0.809419,318.546015,8.264057,1.382773,Turbine_01,51.007408,49.288458,3.342025,48.370728,54171
3,2021-11-11 10:49:00,26.142280,39.120330,33.147231,785.187622,50.829192,20.329997,29.411789,32.939673,1.495129,1.374825,159.353495,3.115742,0.527393,Turbine_01,47.833076,49.288458,3.342025,48.370728,54171
4,2021-07-14 15:08:00,97.355926,102.576240,36.209320,778.595540,51.831118,21.149041,32.748087,36.114233,1.768352,1.642608,148.833809,3.169921,0.521003,Turbine_01,49.604198,49.288458,3.342025,48.370728,54171


In [12]:
df_train_ext['turbine_id'] = df_train_ext['turbine_id'].factorize()[0]
df_test_ext['turbine_id'] = df_test_ext['turbine_id'].factorize()[0]

In [13]:
feats = df_train_ext.select_dtypes("number").columns
target_list = ["Target"]
feats = [x for x in feats if x not in target_list]

In [14]:
feats

['active_power_calculated_by_converter',
 'active_power_raw',
 'ambient_temperature',
 'generator_speed',
 'generator_winding_temp_max',
 'grid_power10min_average',
 'nc1_inside_temp',
 'nacelle_temp',
 'reactice_power_calculated_by_converter',
 'reactive_power',
 'wind_direction_raw',
 'wind_speed_raw',
 'wind_speed_turbulence',
 'turbine_id',
 'mean',
 'std',
 'median',
 'size']

In [15]:
X_train = df_train_ext[feats].values
y_train = df_train["Target"].values
X_test = df_test_ext[feats].values

In [16]:
df_train["Target_log"] = np.log( df_train['Target'] )
#df_train['Target_log'].hist(bins=50);

In [17]:
y_train = df_train["Target_log"].values

In [18]:
model = xgb.XGBRegressor(max_depth=7, n_estimators=100,learning_rate=0.1, random_state=0)
model.fit(X_train,y_train)
y_log_pred = model.predict(X_test)

In [19]:
y_pred = np.exp(y_log_pred)

In [20]:
df_test["Target"] = y_pred

In [21]:
df_test[ ["Target"] ].to_csv('../content/drive/MyDrive/ReNew_Participants_Data/simple_submission_log_ext.csv', index=False)